In [1]:
using Pkg; Pkg.activate("/home/dhairyagandhi96/modelzoo/model-zoo/script/../other/bitstring-parity"); Pkg.instantiate(); Pkg.add(PackageSpec(name="CUDAdrv", rev = "master")); Pkg.add(PackageSpec(name="CUDAnative", rev = "master")); Pkg.add(PackageSpec(name="CuArrays", rev = "master")); Pkg.add(PackageSpec(name="Flux", rev = "master"))

include("data.jl")
using Flux, Statistics
using Flux: onehot, onehotbatch, throttle, crossentropy, reset!, onecold
using CuArrays

const epochs = 10

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
  Updating git-repo `https://github.com/JuliaGPU/CUDAdrv.jl.git`
 Resolving package versions...
  Updating `~/modelzoo/model-zoo/other/bitstring-parity/Project.toml`
 [no changes]
  Updating `~/modelzoo/model-zoo/other/bitstring-parity/Manifest.toml`
 [no changes]
  Updating git-repo `https://github.com/JuliaGPU/CUDAnative.jl.git`
 Resolving package versions...
  Updating `~/modelzoo/model-zoo/other/bitstring-parity/Project.toml`
  [be33ccc6] ~ CUDAnative v0.10.1+ #master (https://github.com/JuliaGPU/CUDAnative.jl.git)
  Updating `~/modelzoo/model-zoo/other/bitstring-parity/Manifest.toml`
  [be33ccc6] ~ CUDAnative v0.10.1+ #master (https://github.com/JuliaGPU/CUDAnative.jl.git)
  Building CUDAnative → `~/.julia/packages/CUDAnative/SvVtC/deps/build.log`
  Updating git-repo `https://github.com/JuliaGPU/CuArrays.jl.git`
 Resolving package versions...
  Updating `~/mode

10

training data of bit strings from length 2 to 10

In [2]:
train = gendata(2000, 1:10)
train = gpu.(train)

2000-element Array{Tuple{Flux.OneHotMatrix{CuArrays.CuArray{Flux.OneHotVector,1}},Flux.OneHotVector},1}:
 ([true false … false true; false true … true false], [false, true])
 ([true true … false false; false false … true true], [true, false])
 ([true true … true true; false false … false false], [true, false])
 ([false true; true false], [false, true])                          
 ([true true false; false false true], [false, true])               
 ([true; false], [true, false])                                     
 ([false true … false false; true false … true true], [true, false])
 ([false true … false true; true false … true false], [false, true])
 ([false true true false; true false false true], [true, false])    
 ([false true … true true; true false … false false], [false, true])
 ⋮                                                                  
 ([true; false], [true, false])                                     
 ([false true false true; true false true false], [true, false])   

validation data of bit strings of length 10

In [3]:
val = gendata(100, 10)
val = gpu.(val)

scanner = LSTM(length(alphabet), 20) |> gpu
encoder = Dense(20, length(alphabet)) |> gpu

function model(x)
    state = scanner.(x.data)[end]
    reset!(scanner)
    softmax(encoder(state))
end

loss(x, y) = crossentropy(model(x), y)

opt = ADAM(params(scanner, encoder))

for i=1:epochs
    Flux.train!(loss, train, opt)
end

┌ Info: Building the CUDAnative run-time library for your sm_70 device, this might take a while...
└ @ CUDAnative /home/dhairyagandhi96/.julia/packages/CUDAnative/SvVtC/src/compiler/rtlib.jl:156
┌ Warning: ADAM(params) is deprecated; use ADAM(η::Float64) instead
│   caller = top-level scope at none:0
└ @ Core none:0
┌ Warning: train!(loss, data, opt) is deprecated; use train!(loss, params, data, opt) instead
│   caller = top-level scope at string:18 [inlined]
└ @ Core ./string:18
┌ Warning: calls to Base intrinsics might be GPU incompatible
│   exception = (CUDAnative.MethodSubstitutionWarning(exp(x::T) where T<:Union{Float32, Float64} in Base.Math at special/exp.jl:75, exp(x::Float32) in CUDAnative at /home/dhairyagandhi96/.julia/packages/CUDAnative/SvVtC/src/device/libdevice.jl:90), Base.StackTraces.StackFrame[exp at exp.jl:75, mapreducedim_kernel_parallel at mapreduce.jl:29])
└ @ CUDAnative /home/dhairyagandhi96/.julia/packages/CUDAnative/SvVtC/src/compiler/irgen.jl:101
┌ Warning: c

sanity test

In [4]:
tx = map(c -> onehotbatch(c, alphabet), [
    [false, true], # 01 -> 1
    [true, false], # 10 -> 1
    [false, false], # 00 -> 0
    [true, true]]) # 11 -> 0
[onecold(model(x)) - 1 for x in tx] |> println

[1, 1, 0, 0]


Try running the model on strings of length 50.

Even though the model has only been trained with
much shorter strings, it has learned the
parity function and will accurate on longer strings.

In [5]:
function t50()
    l = batch_loss(gendata(1000, 50))
    println(l)
end

t50 (generic function with 1 method)